In [13]:
# Cài đặt thư viện statsmodels nếu chưa có
!pip install statsmodels

In [14]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from itertools import product
import pandas as pd
from google.colab import drive

# Kết nối Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error
from itertools import product
import os

class StockModelEvaluation:
    def __init__(self):
        self.results_additive = []
        self.results_multiplicative = []

    def prepare_data(self, df, ticker):
        """
        Chuẩn bị dữ liệu cho một mã chứng khoán bằng cách thiết lập chỉ mục, đổi tên các cột,
        và chọn các cột cần thiết.
        """
        df_ticker = df[df['ticker'] == ticker].set_index('time')
        df_ticker = df_ticker.rename(columns={'close': 'y'})
        df_ticker = df_ticker[['y']]
        return df_ticker

    def optimize_holt_winters(self, train_df, seasonal='add', seasonal_periods=12):
        best_mae = float('inf')
        best_params = None

        alphas = betas = gammas = np.arange(0.1, 1.0, 0.1)
        params = product(alphas, betas, gammas)

        for alpha, beta, gamma in params:
            try:
                model = ExponentialSmoothing(
                    train_df['y'],
                    trend='add',
                    seasonal=seasonal,
                    seasonal_periods=seasonal_periods
                ).fit(
                    smoothing_level=alpha,
                    smoothing_trend=beta,
                    smoothing_seasonal=gamma
                )

                forecast = model.fittedvalues
                mae = mean_absolute_error(train_df['y'], forecast)

                if mae < best_mae:
                    best_mae = mae
                    best_params = (alpha, beta, gamma)

            except Exception as e:
                continue

        return best_params

    def fit_and_predict_holt_winters(self, train_df, test_df, ticker, seasonal_mode='add', seasonal_periods=12):
        best_alpha, best_beta, best_gamma = self.optimize_holt_winters(train_df, seasonal=seasonal_mode, seasonal_periods=seasonal_periods)

        model = ExponentialSmoothing(
            train_df['y'],
            trend='add',
            seasonal=seasonal_mode,
            seasonal_periods=seasonal_periods
        ).fit(
            smoothing_level=best_alpha,
            smoothing_trend=best_beta,
            smoothing_seasonal=best_gamma
        )

        forecast = model.forecast(len(test_df)).rename('Holt_Winters_Forecast')

        y_true = test_df['y'].values
        y_pred = forecast.values

        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))

        print(f'Mã chứng khoán: {ticker}, MAE: {mae}, RMSE: {rmse}, Seasonal: {seasonal_mode}')

        result = {
            'ticker': ticker,
            'mae': mae,
            'rmse': rmse,
            'seasonality_mode': seasonal_mode
        }

        if seasonal_mode == 'add':
            self.results_additive.append(result)
        else:
            self.results_multiplicative.append(result)

        return forecast, mae, rmse

    def plot_forecast_holt_winters(self, train_df, forecast, test_df, ticker, seasonality_mode):
        plt.figure(figsize=(10, 6))
        plt.plot(train_df.index, train_df['y'], label='Train', color='blue')
        plt.plot(test_df.index, test_df['y'], label='Test', color='green')
        plt.plot(test_df.index, forecast, label='Prediction', color='orange')
        plt.title(f'Mã chứng khoán: {ticker} - Holt-Winters Seasonality: {seasonality_mode}')
        plt.xlabel('Ngày')
        plt.ylabel('Giá đóng cửa')
        plt.legend()

        # Tự động lưu ảnh
        filename = f"{ticker}_Holt_Winters_{seasonality_mode}.png"
        plt.savefig(filename)
        plt.show()

    def get_results(self):
        results_additive_df = pd.DataFrame(self.results_additive)
        results_multiplicative_df = pd.DataFrame(self.results_multiplicative)
        return results_additive_df, results_multiplicative_df

    def save_results_to_csv(self, results_df, filename):
        results_df.to_csv(filename, index=False)

    def calculate_average_metrics(self, results_df):
        average_mae = results_df['mae'].mean()
        average_rmse = results_df['rmse'].mean()
        return average_mae, average_rmse






In [17]:
# Đường dẫn tới file trên Google Drive
train_file_path = '/content/drive/MyDrive/LuanVan/data/ComVN30_train.csv'
test_file_path = '/content/drive/MyDrive/LuanVan/data/ComVN30_test.csv'

# Đọc dữ liệu
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)

# Chuyển đổi cột 'time' sang định dạng datetime
train_df['time'] = pd.to_datetime(train_df['time'])
test_df['time'] = pd.to_datetime(test_df['time'])

In [18]:
import warnings
warnings.filterwarnings("ignore")

# Cách sử dụng
evaluator_holt_winters = StockModelEvaluation()
tickers = train_df['ticker'].unique()


In [19]:

# Xử lý Holt-Winters Additive
for ticker in tickers:
    print(f"Đang xử lý mã chứng khoán (Cộng): {ticker}")

    train_df_ticker = evaluator_holt_winters.prepare_data(train_df, ticker)
    test_df_ticker = evaluator_holt_winters.prepare_data(test_df, ticker)

    forecast_additive, mae_additive, rmse_additive = evaluator_holt_winters.fit_and_predict_holt_winters(
        train_df_ticker, test_df_ticker, ticker, seasonal_mode='add'
    )
    evaluator_holt_winters.plot_forecast_holt_winters(train_df_ticker, forecast_additive, test_df_ticker, ticker, 'Additive')

# Tính toán trung bình cho Additive
results_df_additive, _ = evaluator_holt_winters.get_results()
average_mae_add, average_rmse_add = evaluator_holt_winters.calculate_average_metrics(results_df_additive)
print(f"MAE Trung bình (Additive): {average_mae_add}, RMSE Trung bình (Additive): {average_rmse_add}")

# Lưu kết quả Additive vào CSV
evaluator_holt_winters.save_results_to_csv(results_df_additive, "holt_winters_additive_results.csv")


Output hidden; open in https://colab.research.google.com to view.

In [20]:
# Xử lý Holt-Winters Multiplicative
for ticker in tickers:
    print(f"Đang xử lý mã chứng khoán (Nhân): {ticker}")

    train_df_ticker = evaluator_holt_winters.prepare_data(train_df, ticker)
    test_df_ticker = evaluator_holt_winters.prepare_data(test_df, ticker)

    forecast_multiplicative, mae_multiplicative, rmse_multiplicative = evaluator_holt_winters.fit_and_predict_holt_winters(
        train_df_ticker, test_df_ticker, ticker, seasonal_mode='mul'
    )
    evaluator_holt_winters.plot_forecast_holt_winters(train_df_ticker, forecast_multiplicative, test_df_ticker, ticker, 'Multiplicative')

# Tính toán trung bình cho Multiplicative
_, results_df_multiplicative = evaluator_holt_winters.get_results()
average_mae_mul, average_rmse_mul = evaluator_holt_winters.calculate_average_metrics(results_df_multiplicative)
print(f"MAE Trung bình (Multiplicative): {average_mae_mul}, RMSE Trung bình (Multiplicative): {average_rmse_mul}")

# Lưu kết quả Multiplicative vào CSV
evaluator_holt_winters.save_results_to_csv(results_df_multiplicative, "holt_winters_multiplicative_results.csv")


Output hidden; open in https://colab.research.google.com to view.